In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, set_seed
from datasets import *
import numpy as np
import torch
import ast

In [2]:
SEED = 42
set_seed(SEED)

# any combination of these years and dataset types can be used
# year = 2020
# year = 2022
# dataset_type = "full"
# dataset_type = "smaller"

# only combination of these years and dataset types can be used
year = 2022
dataset_type = "handwritten"

# only combination of these years and dataset types can be used
# year = 2042 # idk some random number for file names
# dataset_type = "full-combined"
# dataset_type = "smaller-combined"

local_models_path = '../../data/models/T5'

model_name = "mrm8488/t5-small-finetuned-squadv2" # small model
# model_name = "mrm8488/t5-base-finetuned-squadv2"

Reference notebooks:

https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/T5_on_TPU.ipynb#scrollTo=KdmKlMkfcLa0

https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb

### Load tokenizer and model

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


### Load the dataset

In [4]:
# Load the dataset from file and split it into train and test datasets
if dataset_type == "full":
    data = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-{year}-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
elif dataset_type == "smaller":
    data = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-{year}-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
    data["train"] = data["train"].select(range(len(data["train"]) // 2))
elif dataset_type == "full-combined":
    data = load_dataset('csv', data_files="../../data/clean/sustainability-report-2042-squad-format.csv",
                        delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=SEED)
elif dataset_type == "smaller-combined":
    data = load_dataset('csv', data_files="../../data/clean/sustainability-report-2042-squad-format.csv",
                        delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=SEED)
    data["train"] = data["train"].select(range(len(data["train"]) // 2))
elif dataset_type == "handwritten":
    data = load_dataset('csv', data_files=f"../../data/clean/QA_SR_2022_Expert-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
else:
    raise Exception("Invalid dataset type")

Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-853b320bab41342e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-690d708f98f9a3b4.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b3c2cbaa1563558c.arrow


In [5]:
# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data["test"] = data["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["test"] = data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data["test"] = data["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data["test"] = data["test"].remove_columns(["text", "answer_start"])

data["train"] = data["train"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["train"] = data["train"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                  "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data["train"] = data["train"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data["train"] = data["train"].remove_columns(["text", "answer_start"])

Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-227bc1b6d7ce66b1.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ea73fd3146b3fd5a.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-51580e58cadacd1e.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-4135a5cc2b39b525.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec

In [6]:
data["train"][0]

{'question': 'How many new green loan offers did NLB introduce for legal entities? ',
 'context': 'NLB introduced 3 new green loan offers for legal entities (Micro and SME): • NLB Green partner loan to finance the construction of turnkey solar power plants • NLB Green Loan for investments in the energy efficiency of business buildings • NLB Green Loan for reducing the carbon footprint, that cover the area of energy efficiency, circular economy and sustainable agriculture',
 'answers': {'answer_start': [0],
  'text': ['NLB introduced 3 new green loan offers for legal entities (Micro and SME)']}}

### Tokenize the dataset

In [7]:
# process the examples in input and target text format and the eos token at the end
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (
        example['question'], example['context'])
    example['target_text'] = '%s </s>' % example['answers']['text'][0]
    return example

# tokenize the examples
def convert_to_features(examples):
    model_inputs = tokenizer(
        examples['input_text'], pad_to_max_length=True, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['target_text'], pad_to_max_length=True, max_length=128, truncation=True)
        temp = np.array(labels["input_ids"])
        temp[temp == tokenizer.pad_token_id] = -100
        labels["input_ids"] = temp.tolist()

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
train_data, test_data = data["train"], data["test"]

train_data = train_data.map(add_eos_to_examples, load_from_cache_file=False)
train_data = train_data.map(
    convert_to_features, batched=True, load_from_cache_file=False)

test_data = test_data.map(add_eos_to_examples, load_from_cache_file=False)
test_data = test_data.map(
    convert_to_features, batched=True, load_from_cache_file=False)

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\transformers\tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [9]:
train_data[0]

{'question': 'How many new green loan offers did NLB introduce for legal entities? ',
 'context': 'NLB introduced 3 new green loan offers for legal entities (Micro and SME): • NLB Green partner loan to finance the construction of turnkey solar power plants • NLB Green Loan for investments in the energy efficiency of business buildings • NLB Green Loan for reducing the carbon footprint, that cover the area of energy efficiency, circular economy and sustainable agriculture',
 'answers': {'answer_start': [0],
  'text': ['NLB introduced 3 new green loan offers for legal entities (Micro and SME)']},
 'input_text': 'question: How many new green loan offers did NLB introduce for legal entities?   context: NLB introduced 3 new green loan offers for legal entities (Micro and SME): • NLB Green partner loan to finance the construction of turnkey solar power plants • NLB Green Loan for investments in the energy efficiency of business buildings • NLB Green Loan for reducing the carbon footprint, th

### Fine tunning

In [10]:
name = model_name.split("/")[-1]
output_dir = f"{local_models_path}/{name}-finetuned-NLB-QA-{year}-{dataset_type}"
if "small" in model_name:
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=25,
        predict_with_generate=True,
        fp16=True,
        push_to_hub=False,
        load_best_model_at_end=True
    )
elif "base" in model_name:
    # with this batch size the base model fits on a GPU with 8GB of memory
    training_args = Seq2SeqTrainingArguments(
        output_dir = output_dir,
        evaluation_strategy = "epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=25,
        predict_with_generate=True,
        fp16=True,
        push_to_hub=False,
        load_best_model_at_end=True
    )
else:
    raise ValueError("Model name not supported")

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [11]:
trainer.train()
trainer.save_model(output_dir)

c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/150 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.0605270862579346, 'eval_runtime': 0.1256, 'eval_samples_per_second': 151.273, 'eval_steps_per_second': 23.885, 'epoch': 1.0}


...

{'eval_loss': 0.6240019798278809, 'eval_runtime': 0.1237, 'eval_samples_per_second': 153.616, 'eval_steps_per_second': 24.255, 'epoch': 23.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.6219844818115234, 'eval_runtime': 0.1249, 'eval_samples_per_second': 152.141, 'eval_steps_per_second': 24.022, 'epoch': 24.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.6207030415534973, 'eval_runtime': 0.1189, 'eval_samples_per_second': 159.831, 'eval_steps_per_second': 25.237, 'epoch': 25.0}
{'train_runtime': 48.6011, 'train_samples_per_second': 22.119, 'train_steps_per_second': 3.086, 'train_loss': 0.6681754048665365, 'epoch': 25.0}
